In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from datetime import datetime
import pandas as pd
import os
import time

In [31]:
class UserSettings:
    # Инициализация объекта
    def __init__(self, csv_file_path):
        
        self.csv_file_path = csv_file_path  # указываем путь к файлу
        self.name = os.path.basename(csv_file_path).split('.')[0]

        self.df = pd.read_csv(self.csv_file_path, header=None, delimiter=';') # читаем файл
        self.array_2d = self.df.values  # создаем двумерный массив для строк

        self.titles = [] # пустой список названий столбцов
        for i in range(len(self.array_2d[0])):       # цикл для чтения селекторов первой веб-страницы
            self.titles.append(self.array_2d[0][i])

        self.web_page_url = self.array_2d[1][0] # присваиваем первое значение второй строки как ссылку на веб-страницу

        self.web_element = []   # указываем на конкретный веб-элемент страницы
        for i in range(len(self.array_2d[3])):
            self.web_element.append(self.array_2d[2][i])

        self.css_selectors = [] # пустой список селекторов
        for i in range(len(self.array_2d[3])):       # цикл для чтения селекторов первой веб-страницы
            self.css_selectors.append(self.array_2d[3][i])      # заполняем пустой список

        self.tags = []  # указываем теги классов
        for i in range(len(self.array_2d[4])):
            self.tags.append(self.array_2d[4][i])

        self.conditions = []    # перечисляем условия заполнения
        for i in range(len(self.array_2d[5])):
            self.conditions.append(self.array_2d[5][i])
    
    def get_csv_file_path(self):    # получить путь к файлу
        return self.csv_file_path
    
    def get_titles(self):       # получить заголовки
        return self.titles
    
    def get_web_page_url(self):     # получить url страницы
        return self.web_page_url
    
    def get_web_element(self):      # получить веб-элемент
        return self.web_element

    def get_css_selectors(self):    # получить селекторы
        return self.css_selectors
    
    def get_tags(self):     # получить теги
        return self.tags
    
    def get_conditions(self):   # получить условия
        return self.conditions

    def get_file_name(self):        # получить название файла
        return self.name

In [32]:
class DataParser:

    def __init__(self, web_page_url, web_element, titles, css_selectors, tags, conditions):
        self.web_page_url = web_page_url
        self.web_element = web_element
        self.titles = titles
        self.css_selectors = css_selectors
        self.tags = tags
        self.conditions = conditions

    def extract_data(self):
        #data = {'Title': [], 'Year': [], 'Duration': [], 'Age': []}
        data = {name: [] for name in self.titles}

        driver = webdriver.Edge()

        driver.get(self.web_page_url)
        if driver.page_source:
            self.html_content = driver.page_source
            print("Successful")
        else:
            print("Error: couldn't fetch HTML content")

        wait = WebDriverWait(driver, 10)
        metadata_list = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"{self.web_element[0]}.{self.web_element[1]}")))
        print("METADATA_LIST:\n", metadata_list)
        for selector,tag in zip(self.css_selectors, self.tags):         # соответствие индексов тега и селектора для подстановки
            items = metadata_list.find_elements(By.CSS_SELECTOR, f"{tag}.{selector}")
            for item in items:
                for condition, title in zip(self.conditions, self.titles):  # соответствие условия к заголовку для подстановки
                    if eval(condition):     # вставка кода из документа в программу (опасн)
                        data[f'{title}'].append(item.text.strip())

        # Получаем максимальную длину списка среди всех столбцов
        max_length = max(len(data[col]) for col in data)
        print("DATA_OUT:\n", data)
        # Заполняем отсутствующие значения в более коротких столбцах значением по умолчанию (например, '')
        for col in data:
            data[col] += [''] * (max_length - len(data[col]))
        result = pd.DataFrame(data)
        print("DF_RESULT:\n", result)
        driver.quit()
        return result

In [34]:
filename = input("Enter a file-instruction name: ")     # Вводим название файла с расширение (filename.csv)
user_settings = UserSettings(f"D:\ISproject\dataStorage\{filename}")    # Передача файла в класс пользовательских настроек

folder_path = f"D:\\ISproject\\dataStorage\\{user_settings.get_file_name()}"    # Создание каталога для переданного файла
os.makedirs(folder_path, exist_ok=True)

data_collector = DataParser(user_settings.get_web_page_url(), user_settings.get_web_element(), user_settings.get_titles(), user_settings.get_css_selectors(), user_settings.get_tags(), user_settings.get_conditions())
result = data_collector.extract_data()  # Сбор данных

time = datetime.now().strftime("%Y%m%d_%H%M")   # Сохранение полученных данных с указанием времени и этапа выполнения
result.to_csv(f"{folder_path}\\{user_settings.get_file_name()}_{time}_parsed.csv", index=False, sep=';')

Successful
METADATA_LIST:
 <selenium.webdriver.remote.webelement.WebElement (session="9df1766718ca8b33eb0b556fe3c2f01c", element="f.AB2F6C55B07B300323360942981FE765.d.9248BC7AFFDECA864E6D66DC60918946.e.27")>
DATA_OUT:
 {'Title': ['1. Побег из Шоушенка', '2. Крёстный отец', '3. Тёмный рыцарь', '4. Крёстный отец 2', '5. 12 рaзгневанных мужчин', '6. Список Шиндлера', '7. Властелин колец: Возвращение короля', '8. Криминальное чтиво', '9. Властелин колец: Братство кольца', '10. Хороший, плохой, злой', '11. Форрест Гамп', '12. Властелин колец: Две крепости', '13. Бойцовский клуб', '14. Начало', '15. Звёздные войны. Эпизод 5: Империя наносит ответный удар', '16. Матрица', '17. Славные парни', '18. Пролетая над гнездом кукушки', '19. Семь', '20. Интерстеллар', '21. Дюна: Часть вторая', '22. Эта замечательная жизнь', '23. Семь самураев', '24. Молчание ягнят', '25. Спасти рядового Райана', '26. Город Бога', '27. Жизнь прекрасна', '28. Зелёная миля', '29. Терминатор 2: Судный день', '30. Звёздные